In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import re
import functools
from time import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import utils
import data_reader
from batch_norm import Batch_Normalizer
from model import CNN_Model

In [59]:
# File parameters
DATA_DIR = 'data'
TRAIN_IMAGES = 'data_train.bin'
TRAIN_TEMPLATES = 'fv_train.bin'
VALID_IMAGES = 'data_valid.bin'
VALID_TEMPLATES = 'fv_valid.bin'
TEST_IMAGES = 'data_test.bin'

# Data parameters
NUM_TRAIN_IMAGES = 100000
NUM_VALID_IMAGES = 10000
NUM_TEST_IMAGES = 10000
IMAGE_DIM = 48
TEMPLATE_DIM = 128

# Reader parameters
BATCH_SIZE = 100
VALID_BATCH_SIZE = 1000
NUM_READ_THREADS = 1
MIN_AFTER_DEQUEUE = 10000
CAPACITY = 11000

# Batch norm parameters
EMA_DECAY = 0.99
BN_EPSILON = 0.01

# Log parameters
LOG_DIR = 'logs'
SAVE_DIR = 'checkpoints'

In [78]:
model = CNN_Model('run1', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
                  VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

model.add_conv('conv_1', ksize=[3, 3, 1, 10])
model.add_batch_norm('bn_1_1')
model.add_conv('res_conv_1', ksize=[3, 3, 10, 10], residual=-3)
model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1], dropout=0)
model.add_batch_norm('bn_1_2')

model.add_conv('conv_2', ksize=[3, 3, 10, 20])
model.add_batch_norm('bn_2_1')
model.add_conv('res_conv_2', ksize=[3, 3, 20, 20], residual=-3)
model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
model.add_batch_norm('bn_2_2')

model.add_conv('conv_3', ksize=[3, 3, 20, 40])
model.add_batch_norm('bn_3_1')
model.add_conv('res_conv_3', ksize=[3, 3, 40, 40], residual=-3)
model.add_pool('max_pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
model.add_batch_norm('bn_3_2')

model.add_pool('avg_pool', 'avg', ksize=[1, 3, 3, 1], stride=[1, 3, 3, 1])
model.add_fully_connected('fc', size=128)

model.add_mse_loss('mse_loss')
model.add_adam_optimizer('optimizer', init_learning_rate=0.01, decay=True)
model.add_summaries('summaries')

utils.add_description("""
model = CNN_Model('run1', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
                  VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

model.add_conv('conv_1', ksize=[3, 3, 1, 10])
model.add_batch_norm('bn_1_1')
model.add_conv('res_conv_1', ksize=[3, 3, 10, 10], residual=-3)
model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1], dropout=0)
model.add_batch_norm('bn_1_2')

model.add_conv('conv_2', ksize=[3, 3, 10, 20])
model.add_batch_norm('bn_2_1')
model.add_conv('res_conv_2', ksize=[3, 3, 20, 20], residual=-3)
model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
model.add_batch_norm('bn_2_2')

model.add_conv('conv_3', ksize=[3, 3, 20, 40])
model.add_batch_norm('bn_3_1')
model.add_conv('res_conv_3', ksize=[3, 3, 40, 40], residual=-3)
model.add_pool('max_pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
model.add_batch_norm('bn_3_2')

model.add_pool('avg_pool', 'avg', ksize=[1, 3, 3, 1], stride=[1, 3, 3, 1])
model.add_fully_connected('fc', size=128)

model.add_mse_loss('mse_loss')
model.add_adam_optimizer('optimizer', init_learning_rate=0.01, decay=True)
model.add_summaries('summaries')

total parameters = %d
""" % utils.num_parameters(model), model.save_dir, model.name)
print('Total parameters : %d' % utils.num_parameters(model))

Description added at checkpoints\run1\run1_description.txt
Total parameters : 50098


In [79]:
model.initialize_session(restore=True)
model.train(n_batches=10000, step_size=200, batch_size=100, valid_batch_size=1000, save=True)

Batch 200:
--> Train loss = 0.059995
--> Valid loss = 0.171760
Saving session checkpoints\run1\model.ckpt-200
Batch 400:
--> Train loss = 0.042756
--> Valid loss = 0.073106
Saving session checkpoints\run1\model.ckpt-400
Batch 600:
--> Train loss = 0.039257
--> Valid loss = 0.063488
Saving session checkpoints\run1\model.ckpt-600
Batch 800:
--> Train loss = 0.037264
--> Valid loss = 0.062447
Saving session checkpoints\run1\model.ckpt-800
Batch 1000:
--> Train loss = 0.036172
--> Valid loss = 0.056223
Saving session checkpoints\run1\model.ckpt-1000
Batch 1200:
--> Train loss = 0.035081
--> Valid loss = 0.057141
Saving session checkpoints\run1\model.ckpt-1200
Batch 1400:
--> Train loss = 0.034274
--> Valid loss = 0.051856
Saving session checkpoints\run1\model.ckpt-1400
Batch 1600:
--> Train loss = 0.033707
--> Valid loss = 0.052485
Saving session checkpoints\run1\model.ckpt-1600
Batch 1800:
--> Train loss = 0.033230
--> Valid loss = 0.055147
Saving session checkpoints\run1\model.ckpt-1800


KeyboardInterrupt: 

In [80]:
model.close_session()